In [27]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial import ConvexHull
from shapely.geometry import Polygon, Point
import random

points = [Point(random.randint(0, 100), random.randint(0, 100)) for _ in range(30)]

# Удаление дубликатов
unique_points = []
seen = set()
for p in points:
    if (p.x, p.y) not in seen:
        seen.add((p.x, p.y))
        unique_points.append(p)
points = unique_points
# 1. Создаем множества точек E1 и E
# Множество E1 - точки в прямоугольнике [0,3]x[0,3]
E1 = points[0:len(points)//2]
# Множество E2 - точки в прямоугольнике [2,5]x[2,5]
E2 = points[len(points)//2+1:]

# 2. Строим выпуклые оболочки
hull_E1 = ConvexHull(E1)
hull_E2 = ConvexHull(E2)

# 3. Создаем полигоны для выпуклых оболочек
poly_E1 = Polygon(E1[hull_E1.vertices])
poly_E2 = Polygon(E2[hull_E2.vertices])

# 4. Находим пересечение выпуклых оболочек
intersection = poly_E1.intersection(poly_E2)

# 5. Находим внутренние точки
inner_points_E1 = [p for p in E1 if intersection.contains(Point(p))]
inner_points_E2 = [p for p in E2 if intersection.contains(Point(p))]

# 6. Визуализация
plt.figure(figsize=(10, 8))

# Отображаем точки множеств
plt.scatter(E1[:,0], E1[:,1], c='blue', label='Множество E1')
plt.scatter(E2[:,0], E2[:,1], c='green', label='Множество E2')

# Отображаем выпуклые оболочки
for simplex in hull_E1.simplices:
    plt.plot(E1[simplex, 0], E1[simplex, 1], 'b--')
for simplex in hull_E2.simplices:
    plt.plot(E2[simplex, 0], E2[simplex, 1], 'g--')

# Отображаем пересечение
if not intersection.is_empty:
    if intersection.geom_type == 'Polygon':
        x, y = intersection.exterior.xy
        plt.fill(x, y, alpha=0.3, fc='red', label='Пересечение P')
    elif intersection.geom_type == 'MultiPolygon':
        for poly in intersection.geoms:
            x, y = poly.exterior.xy
            plt.fill(x, y, alpha=0.3, fc='red')

# Отображаем внутренние точки
if inner_points_E1:
    inner_E1 = np.array(inner_points_E1)
    plt.scatter(inner_E1[:,0], inner_E1[:,1], c='blue', s=100, marker='*', label='Внутренние точки E1')
if inner_points_E2:
    inner_E2 = np.array(inner_points_E2)
    plt.scatter(inner_E2[:,0], inner_E2[:,1], c='green', s=100, marker='*', label='Внутренние точки E2')

plt.title('Выпуклые оболочки и их пересечение')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid(True)
plt.legend()
plt.show()

# Выводим информацию о внутренних точках
print("Внутренние точки множества E1:")
for p in inner_points_E1:
    print(f"({p[0]:.2f}, {p[1]:.2f})")

print("\nВнутренние точки множества E2:")
for p in inner_points_E2:
    print(f"({p[0]:.2f}, {p[1]:.2f})")

TypeError: float() argument must be a string or a real number, not 'Point'